In [1]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.keras.backend.clear_session()
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

In [3]:
class MyLayer(layers.Layer):
  def __init__(self, input_dim=32, unit=32):
    super(MyLayer, self).__init__()
    
    w_init = tf.random_normal_initializer()
    self.weight = tf.Variable(initial_value=w_init(
            shape=(input_dim, unit), dtype=tf.float32), trainable=True)
    
    b_init = tf.zeros_initializer()
    self.bias = tf.Variable(initial_value=b_init(
            shape=(unit,), dtype=tf.float32), trainable=True)
  
  def call(self, inputs):
    return tf.matmul(inputs, self.weight) + self.bias

x = tf.ones((3, 5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)

tf.Tensor(
[[ 0.01905325  0.04281903 -0.04288897 -0.03219837]
 [ 0.01905325  0.04281903 -0.04288897 -0.03219837]
 [ 0.01905325  0.04281903 -0.04288897 -0.03219837]], shape=(3, 4), dtype=float32)


In [4]:
class MyLayer(layers.Layer):
  def __init__(self, input_dim=32, unit=32):
    super(MyLayer, self).__init__()
    self.weight = self.add_weight(shape=(input_dim, unit),
                        initializer=keras.initializers.RandomNormal(),
                        trainable=True)
    self.bias = self.add_weight(shape=(unit,),
                        initializer=keras.initializers.Zeros(),
                        trainable=True)
  
  def call(self, inputs):
    return tf.matmul(inputs, self.weight) + self.bias

x = tf.ones((3, 5))
my_layer = MyLayer(5, 4)
out = my_layer(x)
print(out)

tf.Tensor(
[[ 0.02579547  0.00582175 -0.02458467 -0.0204228 ]
 [ 0.02579547  0.00582175 -0.02458467 -0.0204228 ]
 [ 0.02579547  0.00582175 -0.02458467 -0.0204228 ]], shape=(3, 4), dtype=float32)


In [5]:
class AddLayer(layers.Layer):
    def __init__(self, input_dim=32):
        super(AddLayer, self).__init__()
        # 只存储，不训练的变量
        self.sum = self.add_weight(shape=(input_dim,),
                                     initializer=keras.initializers.Zeros(),
                                     trainable=False)
       
    
    def call(self, inputs):
        self.sum.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.sum
        
x = tf.ones((3,3))
my_layer = AddLayer(3)
out = my_layer(x)
print(out.numpy())
out = my_layer(x)
print(out.numpy())
print('weight:', my_layer.weights)
print('non-trainable weight:', my_layer.non_trainable_weights)
print('trainable weight:', my_layer.trainable_weights)

[3. 3. 3.]
[6. 6. 6.]
weight: [<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>]
non-trainable weight: [<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([6., 6., 6.], dtype=float32)>]
trainable weight: []


In [6]:
class MyLayer(layers.Layer):
    def __init__(self, unit=32):
        super(MyLayer, self).__init__()
        self.unit = unit
        
    def build(self, input_shape):
        # 在build时获取input_shape
        self.weight = self.add_weight(shape=(input_shape[-1], self.unit),
                                     initializer=keras.initializers.RandomNormal(),
                                     trainable=True)
        self.bias = self.add_weight(shape=(self.unit,),
                                   initializer=keras.initializers.Zeros(),
                                   trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.weight) + self.bias
        

my_layer = MyLayer(3)
x = tf.ones((3,5))
out = my_layer(x)
print(out)
my_layer = MyLayer(3)

x = tf.ones((2,2))
out = my_layer(x)
print(out)

tf.Tensor(
[[-0.05339011  0.03923885 -0.09293302]
 [-0.05339011  0.03923885 -0.09293302]
 [-0.05339011  0.03923885 -0.09293302]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[ 0.06484456 -0.12487485 -0.07908949]
 [ 0.06484456 -0.12487485 -0.07908949]], shape=(2, 3), dtype=float32)
